In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#for 100% jupyter notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#pandas DataFrame column and row display limits
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.float_format = "{:.2f}".format

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
DIR_PATH = '/kaggle/input/home-credit-default-risk/'

/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv


Some helpful utility functions

In [2]:
def read_input_file(directory_path = DIR_PATH, fileName= 'application_train.csv', displayHead = True, displayInfo = True, displayStats = True, num_rows = None, verbose=True):
    '''
    Function to load the given input table
    
    Input:
        directory_path: str, default = DIR_PATH
            Path of directory in which table is stored
        fileName: str, default = application_train.csv
            Name of csv file containing the table
        displayHead: bool, default = True
            Display head for pandas dataframe
        displayInfo: bool, default = True
            Display info for pandas dataframe
        displayStats: bool, default = True
            Describe function for pandas dataframe
        num_rows: int, default = None
            Limit pandas read for debug mode or dry run
        verbose: bool, default = True
            Whether to keep verbosity or not
    '''
        
    df = pd.read_csv(directory_path + fileName, nrows= num_rows)
    if verbose:
        print("Loaded {}... Dataframe contains {}".format(fileName, len(df)))
    if displayHead:
        display(df.head())
    if displayInfo:
        display(df.info(verbose=True, show_counts = True))
    if displayStats:
        display(df.describe().T)
    return df

In [3]:
def one_hot_encoder(df, nan_as_category = False):
    '''
    Function to do one-hot encoding for categorical columns with get_dummies
    
    Input:
        df: Dataframe
            Input dataframe
        nan_as_category: bool, default = False
            Treat NaN as a category
        
    '''
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns